In [3]:
def crank_nicolson(T0, nt, nx, dt, dx, Tv, alpha, q):
    sigma = alpha * dt / dx**2
    
    A = lhs_operator(len(T0) - 2, sigma)
    T = T0.copy()
    T_mat = np.empty((nx,nt))

    for n in range(nt):
        Tsurf = Tv[n]
        b = rhs_vector(T, sigma, Tsurf, q * dx)                             # Generate RHS
        T[1:-1] = linalg.solve(A, b)                                        # Solve the system
        T[-1] = T[-2] + q * dx                                              # Apply Neumann BC
        T[0] = Tsurf        
        T_mat[:,n] = T
        
    return T, T_mat

In [1]:
def ReturnBedrockTemps(RunPars, HeliumPars):
    nt = HeliumPars['thermnt']
    Tv = np.empty((nt))
    Py = 1
    
    HeliumPars['T_Surf'] = Tv 
    # Set parameters.
    L = RunPars['max_depths'][0] + 1  # length of the rod
    nx = HeliumPars['nz']  # number of points on the rod
    dx = L / (nx - 1)  # grid spacing
    alpha = HeliumPars['alpha_cmhr']  # thermal diffusivity of the rock in cm2 / hour
    q = 0 # temperature gradient at the extremity
    # z = 0 # depth at which we want the temperature (cm) --> aka, we want the surface
    # Set the initial temperature distribution.
    
    T0 = np.zeros(nx)
    T0[:] = RunPars['MAT']
    
    #sigma = 0.5
    #dt = sigma * dx**2 / alpha  # time-step size
    dt = HeliumPars['thermdt']
    
    T, T_mat = crank_nicolson(T0, nt, nx, dt, dx, Tv, alpha, q)
    
    HeliumPars['T_Mat'] = T_mat
    
    if HeliumPars['Normal T Fxn'] == True:
        for n in range(nt):
            t = n / nt
            inner_term = np.pi / alpha * Py
            Tv[n] = RunPars['MAT'] - RunPars['T_AMP'] * np.exp(inner_term) * np.cos((2 * np.pi * t / Py) - z * np.sqrt(inner_term) )    
    
    elif HeliumPars['Normal T Fxn'] == False:
        for n in range(nt):
            t = n / nt
            Tv[n] = RunPars['MAT'] - RunPars['T_AMP'] * np.exp(0) * np.cos(2 * np.pi * t / Py - 0)    
            Tv[4147:4315] = Tv[4146] + 6
    
    return T_mat, HeliumPars

In [ ]:
def lhs_operator(N, sigma):
    # Setup the diagonal of the operator.
    D = np.diag(2.0 * (1.0 + 1.0 / sigma) * np.ones(N))
    # Setup the Neumann condition for the last element.
    D[-1, -1] = 1.0 + 2.0 / sigma
    # Setup the upper diagonal of the operator.
    U = np.diag(-1.0 * np.ones(N - 1), k=1)
    # Setup the lower diagonal of the operator.
    L = np.diag(-1.0 * np.ones(N - 1), k=-1)
    # Assemble the operator.
    A = D + U + L
    return A

In [ ]:
def rhs_vector(T, sigma, surf, qdx):
    b = T[:-2] + 2.0 * (1.0 / sigma - 1.0) * T[1:-1] + T[2:]
    # Set Dirichlet condition.
    b[0] += surf
    # Set Neumann condition.
    b[-1] += qdx
    
    return b

In [ ]:
def diff_calc(T):
    
    Diff = [10**(-3928.1*(1/(T[i])) - 2.9315) for i in range(len(T))]              # cm^^2 * s^-1
    
    return Diff